In [ ]:
import logging
logging.getLogger("pdfminer").setLevel(logging.ERROR)  # ← これで suppress

import os
import glob
import pdfplumber

pdf_dir = "../data/input/pdf/"

# チャンク化関数（1ファイル分）
def chunk_pdf(file_path):
    chunks = []
    with pdfplumber.open(file_path) as pdf:
        for i, page in enumerate(pdf.pages):
            text = page.extract_text()
            if text:
                chunks.append({
                    "chunk_id": f"{file_path}_page_{i+1}",
                    "text": text.strip(),
                    "page": i + 1,
                    "file_path": file_path,
                })
    return chunks

# すべてのPDFを処理してまとめてチャンク化
all_chunks = []
pdf_files = glob.glob(os.path.join(pdf_dir, "*.pdf"))

print(f"PDFファイル数: {len(pdf_files)}")

for file_path in pdf_files:
    chunks = chunk_pdf(file_path)
    all_chunks.extend(chunks)
    print(f"{os.path.basename(file_path)}: {len(chunks)} チャンク")

print(f"\n総チャンク数: {len(all_chunks)}")

if all_chunks:
    print(f"\n📄 page {all_chunks[0]['page']} ({all_chunks[0]['file_path']}) の内容:\n")
    print(all_chunks[0]["text"][:1000])


PDFファイル数: 3


In [ ]:
import pandas as pd

df = pd.DataFrame(chunks)
df[["page", "text"]].head(5)


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

# モデルの読み込み（軽量で日本語対応）
model = SentenceTransformer("intfloat/multilingual-e5-small")

# チャンクからテキストだけ抽出
texts = [chunk["text"] for chunk in all_chunks]

# ベクトル化（正規化あり）
embeddings = model.encode(texts, normalize_embeddings=True)

# ベクトルを各チャンクに追加（NumPy形式）
for chunk, emb in zip(all_chunks, embeddings):
    chunk["embedding"] = emb


In [ ]:
# チャンク数とベクトル次元の確認
print(f"チャンク数: {len(all_chunks)}")
print(f"ベクトル次元: {len(all_chunks[0]['embedding'])}")

# 最初の1件を確認
print("\n最初のチャンクのベクトル（先頭5次元）:")
print(all_chunks[0]["embedding"][:5])


In [ ]:
import faiss
import numpy as np
import pickle
from pathlib import Path
from glob import glob
import pdfplumber
from sentence_transformers import SentenceTransformer

pdf_dir = Path("../data/input/pdf")
pdf_paths = sorted(pdf_dir.glob("*.pdf"))

all_chunks = []

# モデルの準備（日本語対応、軽量）
model = SentenceTransformer("intfloat/multilingual-e5-small")

# 各PDFを処理
for pdf_path in pdf_paths:
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            text = page.extract_text()
            if text:
                all_chunks.append({
                    "chunk_id": f"{pdf_path}_page_{i+1}",
                    "text": text.strip(),
                    "page": i + 1,
                    "file_path": str(pdf_path)
                })

print(f"総チャンク数: {len(all_chunks)}")

# 🧠 ベクトル化
texts = [chunk["text"] for chunk in all_chunks]
embeddings = model.encode(texts, normalize_embeddings=True)
for chunk, emb in zip(all_chunks, embeddings):
    chunk["embedding"] = emb

# FAISSインデックス構築・登録
dim = len(all_chunks[0]["embedding"])
index = faiss.IndexFlatL2(dim)
vecs = np.array([chunk["embedding"] for chunk in all_chunks]).astype("float32")
index.add(vecs)
print(f"FAISSインデックスに登録済み: {index.ntotal} 件")

index_path = Path("data/faiss.index")
meta_path = Path("data/faiss_meta.pkl")
index_path.parent.mkdir(parents=True, exist_ok=True)

faiss.write_index(index, str(index_path))
print(f"インデックス保存: {index_path}")

for chunk in all_chunks:
    chunk["embedding"] = None  # 軽量化

with open(meta_path, "wb") as f:
    pickle.dump(all_chunks, f)
print(f"メタ情報保存: {meta_path}")


In [ ]:
import faiss
import pickle
import numpy as np
from sentence_transformers import SentenceTransformer
import pandas as pd

index = faiss.read_index("data/faiss.index")

with open("data/faiss_meta.pkl", "rb") as f:
    chunks = pickle.load(f)

model = SentenceTransformer("intfloat/multilingual-e5-small")

query = "ディープラーニングについて教えて"

# クエリをベクトル化（正規化あり）
query_vec = model.encode([query], normalize_embeddings=True).astype("float32")

# FAISS検索（上位5件）
k = 5
D, I = index.search(query_vec, k)  # D: 距離、I: インデックス

# 検索結果の整形
results = []
for idx, dist in zip(I[0], D[0]):
    if idx < len(chunks):
        result = {
            "page": chunks[idx]["page"],
            "file_path": chunks[idx]["file_path"],
            "text": chunks[idx]["text"][:150] + "...",  # テキスト冒頭だけ表示
            "score(L2距離)": float(dist)
        }
        results.append(result)

df = pd.DataFrame(results)
df
